In [113]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import functools

def reshape_image(image_path,output_loc):
    img = Image.open(image_path)
    
    # If the image is not a PNG, we convert it to a PNG.
    if img.format.lower() != "png":
        filename = image_path.split(".")
        
        png_img =str()
        for el in filename[:-1]:
            png_img += el
        
        png_img = png_img+".png"
        img.save("."+png_img)
        image_path = "."+png_img
        print("The input image has the extension :"+img.format+". It has been converted to a PNG. The new image is stored with a different file extension : "+image_path)
            
    #img = Image.open(image_path)
    
    # If the input image is too large, we reduce its dimensions.
    # max_height = 250
    # For now, we give all the images the same height
    height = 220
    img = Image.open(image_path).convert("RGBA")
    w_img, h_img = img.size
    
    if h_img != height:
        newsize = (int(height*w_img/min(w_img,h_img)), int(height*h_img/min(w_img,h_img)))
        img = img.resize(newsize)
        img.save(image_path)
        
    return image_path

def crop_digits(image_path, output_loc,closing: bool,bluring: bool,tresholding: bool,erosion: bool):
    # image_path is a string : it indicates the path to where the image is stored.
    # output_loc is a string : it indicates the path to where the cropped digits should be stored.
    """
    This function takes an image containing a number and outputs a set of images
    containing each digit included in the image inputed.
    If the input image is of the following form :
    ---------------
    | 2  3  9  5  |
    ---------------
    then the output would be 4 images containing each a digit :

       -----         -----         -----         -----    
       | 2 |         | 3 |         | 9 |         | 5 |
       -----         -----         -----         -----   
    digit_1.png   digit_2.png   digit_3.png   digit_4.png

    Before using the script, the module cv2 needs to be imported : import cv2
    """
    
    # We check if the input image is not too large and if the image is a PNG.
    # If not, we resize the image, and convert it to a PNG.
    #image_path = reshape_image(image_path,output_loc)
    
    
    """
    GRAYSCALING & INVERTING
    """
    img_original = cv2.imread(image_path)
    img = img_original.copy()
    # convert the image from BGR color space to GRAYSCALE
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #cv2.imwrite("./output/1_number_gray.png", img)
    
    # invert the image
    img = cv2.bitwise_not(img)
    #cv2.imwrite("./output/2_number_inverted.png", img)
    
    """
    CLOSING
    """
    #closing small holes inside the digits
    if closing :
        closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE,cv2.getStructuringElement(cv2.MORPH_RECT,(5,5)))
    
    """
    BLURING
    """
    # Apply Gaussian blurring (then thresholding in the next step) to reveal the characters
    if bluring:
        img = cv2.GaussianBlur(img, (1, 1), 0)
        #cv2.imwrite("./output/3_number_blurred.png", img)
    
    """
    TRESHOLDING
    """
    if tresholding:
        #img = cv2.adaptiveThreshold(img, 255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 45, 15) /*ADAPTIVE TRESHOLDING*/
        #ret,img = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
        ret,img = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        img = cv2.bitwise_not(img)
        #cv2.imwrite("./output/4_number_tresh.png", img)
    
    """
    EROSION
    
    Performs erosion on the image. The basic idea of erosion is just like soil erosion only,
    it erodes away the boundaries of foreground object
    """
    if erosion:
        #erosion_shape = cv2.MORPH_CROSS
        #erosion_shape = cv2.MORPH_ELLIPSE
        erosion_shape = cv2.MORPH_RECT
        kernel = cv2.getStructuringElement(erosion_shape, (5,5))
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
        #cv2.imwrite("./output/5_number_erosion.png", img)
    
    """
    Connected Components Analysis
    """
    # Perform connected components analysis on the thresholded image and
    # initialize the mask to hold only the components we are interested in.
    _, labels = cv2.connectedComponents(img)
    mask = np.zeros(img.shape, dtype="uint8") 
    
    # Set lower bound and upper bound criteria for characters
    total_pixels = img.shape[0] * img.shape[1]
    #lower = total_pixels // 70 # heuristic param, can be fine tuned if necessary
    #upper = total_pixels // 20 # heuristic param, can be fine tuned if necessary
    lower = 0.007*total_pixels
    upper = 0.9*total_pixels
    
    #print("Total pixels = "+str(total_pixels)+" , lower = "+str(lower)+" , upper = "+str(upper))
    
    # Loop over the unique components
    for (i, label) in enumerate(np.unique(labels)):
        # If this is the background label, ignore it
        if label == 0:
            continue
 
        # Otherwise, construct the label mask to display only connected component
        # for the current label
        labelMask = np.zeros(img.shape, dtype="uint8")
        labelMask[labels == label] = 255
        numPixels = cv2.countNonZero(labelMask)
 
        # If the number of pixels in the component is between lower bound and upper bound, 
        # add it to our mask
        if numPixels > lower and numPixels < upper:
            mask = cv2.add(mask, labelMask)
    
    #cv2.imwrite("./output/6_number_mask.png", mask)
    
    """
    DETECTING CONTOURS
    """
    # Find contours and get bounding box for each contour
    contours, hierarchy = cv2.findContours(mask.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    boundingBoxes = [cv2.boundingRect(c) for c in contours]
    
    # The contours returned from findContours do not have any particular order.
    # Hence before we can proceed further, we need to sort the bounding boxes from left to right so that
    # we can read the characters in correct order:
    
    digitCnts = [] # contours of digits will be stored here.
    digits = dict() # Keys of this dictionnary will contain the a abscissa of the images
                    # Values contain the digit's image
                    # digits[x] = image
            
    for x,y,w,h in boundingBoxes:
        # below, img.shape[0] represents the height of the image.
        if w>=5 and h >= 0.3*img.shape[0]: 
            digitCnts.append([x,x+w,y,y+h])
            digits[x] = mask[y:y+h, x:x+w].copy()
    
    """
    FILTERING CONTOURS
    """
    # eleminate an image in case it is located inside another image : keep just the outer/parent image.
    redundant = [] # will contain the keys of redundant images : images to be deleted.
    for box_x_1 in digits:
        # box_x_1 represents the abscissa of the box
        height_1 = digits[box_x_1].shape[0]
        width_1 = digits[box_x_1].shape[1]
        area_1 = height_1*width_1
        
        for box_x_2 in digits:
            height_2 = digits[box_x_2].shape[0]
            width_2 = digits[box_x_2].shape[1]
            area_2 = height_2*width_2
            
            # given information of two images, including abscissas and dimenstions,
            # we check whether one is included inside the other, if that's the case, we remove it
            if box_x_1 <= box_x_2 and box_x_1+width_1 >= box_x_2+width_2 and width_1 >= width_2 and height_1 >= height_2 and area_1 > area_2:
                redundant.append(box_x_2)
    
    # delete the redundant images
    for red in redundant:
        digits.pop(red)
        
    # abscissas contain the abscissa of each box.
    # It serves to order the boxes from the left to the right by increasing values of the abscissa x.
    abscissas = list(digits.keys())
    abscissas.sort()
    
    i=1 # digits count : first digiti will have i=1, second digit will have i=2 ...
    digits_filenames = dict()
    for k in abscissas:
        if digits[k].any():
            cv2.imwrite(output_loc+"/digit_"+str(i)+"_.png", digits[k])
            #print(output_loc+"/digit_"+str(i)+".png")
            i+=1
    
    
        
    

In [121]:
crop_digits("./numbers/number_6.png","./output", closing=True, bluring=True,tresholding=True,erosion=False)